# The Iris Classification Challenge

#### Based on a notebook by [Randal S. Olson](http://www.randalolson.com/)

In this Challenge, you will work through a simple machine learning example from start to finish to see what a typical data analysis pipeline looks like. In addition to providing code examples, we will also discuss good practices for data anaylsis. Note that the code of the notebook runs in your browser, allowing you to interact with it and immediately see the outcomes.

The data used here is a slightly modified version of the [Iris data set](https://archive.ics.uci.edu/ml/datasets/Iris), a famous data set collected by botanist Edward Anderson and popularized by the statistician Ronald Fisher in his classic 1936 paper, "The use of multiple measurements in taxonomic problems". Anderson measured the properties of three different species of iris flowers. The data set contains four features from each sample: the length and width of the petals and sepals, in centimeters. 

<img src="images/iris_petal_sepal.png" style="width: 300px;"/>

## Table of contents

0. [Introduction](#Introduction)

1. [Step 1: Framing the problem](#Step-1:-Framing-the-problem) 

2. [Step 2: Checking the data](#Step-2:-Checking-the-data)

3. [Step 3: Tidying the data](#Step-3:-Tidying-the-data)

4. [Step 4: Exploratory analysis](#Step-4:-Exploratory-analysis)

5. [Step 5: Classification](#Step-5:-Classification)

6. [Step 6: Evaluation with cross-validation](#Step-6:-Evaluation-with-cross-validation)

7. [Sources](#Sources)

## Introduction

[[ go back to the top ]](#Table-of-contents)

For the purposes of this exercise, let's pretend we would like to create a smartphone app that automatically identifies species of flowers from pictures taken on the smartphone. We're working with a team of data scientists and will be building a part of the data analysis pipeline for this app.

Our task is to create a demo machine learning model that **takes four measurements from flowers** (petal length, petal width, sepal length, and sepal width) and **identifies the species based on these measurements** alone. The measurements we're using currently come from hand-measurements by our field researchers, but they will be automated using an image processing model in the future.

To develop the demo, we've been given a [data set](iris-data.csv) which includes measurements for three types of *Iris* flowers: Iris versicolor, Iris setosa, and Iris virginica.

<img src="images/iris-types.png" style="width: 800px;"/>

## Step 1: Framing the problem 

[[ go back to the top ]](#Table-of-contents)

The first step to any data analysis project is to define the question or problem we're trying to solve, and to define a measure (or set of measures) for our success at solving that task. The following set of questions can serve as a useful guide:

>What kind of data analytic question are we trying to answer (e.g. exploration, association causality)?

We're trying to classify the species (i.e., class) of Iris flowers based on four measurements: sepal length, sepal width, petal length, and petal width.

>What is the metric for success?

Since we're performing classification, we can use accuracy — the fraction of correctly classified flowers — to quantify how well our model is performing. We aim to achieve at least 90% accuracy.

>What is the context for the question and the scientific or business application?

We're building a part of the data analysis pipeline for a smartphone app that will be able to classify species of flowers from pictures taken on the smartphone. In the future, this pipeline will be connected to another pipeline that automatically measures from pictures the traits we're using to perform this classification.

>What is the experimental design?

We know that our field researchers have hand-measured 50 randomly-sampled flowers of each species using a standardized methodology. The field researchers take pictures of each flower they sample from pre-defined angles so the measurements and species can be confirmed by the other field researchers at a later point. 

>Can the question be answered with the available data?

The data set we currently have is only for three types of *Iris* flowers. The model trained on this data set will thus also only work for Iris flowers, and we will need more data to create a general flower classifier.

## Step 2: Checking the data

[[ go back to the top ]](#Table-of-contents)

The next step is to look closer at the data we're working with. Spotting potential errors in the data set early can save a lot of time during our later analysis. 

Let's start by reading the data into a pandas DataFrame object and displaying the first five rows. A DataFrame object is a 2-dimensional, labeled data structure with columns that can be of different types; you can think of it like of a spreadsheet.

**Remember to press *Shift+Enter* to run each code cell.**

In [ ]:
# The first line below loads the pandas package, which provides the DataFrame structure; 
# an overview of the packages used in this notebook is provided in the last section
import pandas as pd

# Load data from file
iris_data = pd.read_csv('iris-data.csv')

# Display the first five rows
iris_data.head()

The first row in the data file defines the column headers, which indicate which measurement each column represents and the unit of measurement. Each row below the header row represents an entry for a flower: four measurements and one class, which tells us the species of the flower.

One of the first things we should look for is **missing data.** Thankfully, the field researchers already told us that they put a 'NA' into the spreadsheet when they were missing a measurement. We can tell pandas to automatically interpret values with 'NA' as missing values:

In [ ]:
iris_data = pd.read_csv('iris-data.csv', na_values=['NA'])

Next, it's always a good idea to look at the distribution of the data — especially to identify potential **outliers**. Let's start by printing out some summary statistics about the numerical attributes of the data set:

In [ ]:
iris_data.describe()

We can get some useful pieces of information from this table. For example, we observe that five `petal_width_cm` entries are missing. However, identifying outliers and errors in a large table of numbers is difficult; some visualization of the data would be helpful. 

For this, let's set up the notebook so we can plot inside of it.

In [ ]:
# This line tells the notebook to show plots inside of the notebook
%matplotlib inline

# Load the plotting libraries
import matplotlib.pyplot as plt
import seaborn as sb

A quick way to obtain an overview of the data is to plot **histograms** for the numerical attributes. A histogram shows the number of data points (on the vertical axis) for which a selected attribute falls in a given value range (on the horizontal axis). Here is the histogram of the petal length attribute:

In [ ]:
iris_data['petal_length_cm'].hist()
plt.xlabel('petal_length_cm')
plt.ylabel('number of samples')
;

Another useful way to visualize data are **scatterplots**. A scatterplot displays the relationship between two numerical attributes by showing the data as a collection of points; the x- and y-coordinate of each point are the values of the two selected attributes for that point. In the following plot, we use petal length and petal width as attributes, and additionaly color each point to indicate the class it belongs to:

In [ ]:
sb.scatterplot(x="petal_length_cm", y="petal_width_cm",data=iris_data,hue='class')
;

### <font color=blue>**It's your turn**</font> 

Pick two attributes and plot your own scatterplot!


In [ ]:
# Type your code here


Next, let's combine histograms and scatterplots to create a **scatterplot matrix**. Scatterplot matrices plot the histograms of each column along the diagonal, and a matrix of scatterplots for the combination of any two attributes. They make for an efficient tool to look for errors in our data.

In [ ]:
# Note: We temporarily drop the rows with 'NA' values before plotting
# because the plotting function does not know how to handle them
sb.pairplot(iris_data.dropna())
;

The scatterplot matrix becomes even more informative if we again use colors to indicate the classes, allowing us see some further issues with the data set:

In [ ]:
sb.pairplot(iris_data.dropna(),hue='class')
;

### <font color=blue>**It's your turn**</font> 

Take a moment to analyse the charts: Which potential issues do you notice? Write them down in the cell below before moving on to the next step.

- ...
- ... 
- ...

## Step 3: Tidying the data

[[ go back to the top ]](#Table-of-contents)

So far, we could observe several issues with our data set:

1. There are five classes when there should only be three, likely indicating some labeling errors.

2. There are some clear outliers in the measurements that may be erroneous: one `sepal_width_cm` entry for `Iris-setosa` falls well outside its normal range; and several `sepal_length_cm` entries for `Iris-versicolor` are near-zero for some reason.

3. We have some rows with missing values.

In all these cases, we need to figure out what to do with the erroneous data. Let's walk through the issues one by one and fix them before proceeding with the analysis. 

#### 1. There are five classes when there should only be three.

After talking with the field researchers, it sounds like one of them forgot to add `Iris-` before their `Iris-versicolor` entries. The other extraneous class, `Iris-setossa`, was simply a typo that they forgot to fix.

We can easily fix these errors in the DataFrame:

In [ ]:
# Renaming the mislabeled classes
iris_data.loc[iris_data['class'] == 'versicolor', 'class'] = 'Iris-versicolor'
iris_data.loc[iris_data['class'] == 'Iris-setossa', 'class'] = 'Iris-setosa'

# Checking that only the correct three class types remain
iris_data['class'].unique()

#### 2.a There are some clear outliers that may be erroneous: one `sepal_width_cm` entry for `Iris-setosa` well outside its normal range.

Fixing outliers can be tricky, because it's rarely clear whether the outlier was caused by measurement error, recording the data in improper units, or if the outlier is a real anomaly. If we decide to exclude any data, we need to make sure to document what data we excluded and provide solid reasoning for excluding that data. 

In the case of the one anomalous `sepal_width_cm` entry for `Iris-setosa`, let's say our field researchers know that it's impossible for `Iris-setosa` to have a sepal width below 2.5 cm. We therefore decide to remove that specific entry:

In [ ]:
# We keep only 'Iris-setosa' rows for which the sepal width is equal or greater than 2.5 cm
iris_data = iris_data.loc[(iris_data['class'] != 'Iris-setosa') | (iris_data['sepal_width_cm'] >= 2.5)]

iris_data.loc[iris_data['class'] == 'Iris-setosa', 'sepal_width_cm'].hist()
plt.xlabel('sepal_width_cm for Iris-setosa')
plt.ylabel('number of samples')
;

#### 2.b There are some clear outliers that may be erroneous: several `sepal_length_cm` entries for `Iris-versicolor` are near-zero.

Let's display the rows containing the suspect entries and take a look at the distribution of sepal lengths for `Iris-versicolor`:

In [ ]:
# Plot the histogram
iris_data.loc[iris_data['class'] == 'Iris-versicolor', 'sepal_length_cm'].hist()
plt.xlabel('sepal_length_cm for Iris-versicolor')
plt.ylabel('number of samples')


# Display the Iris-versicolor rows with a sepal length below 1 cm
iris_data.loc[(iris_data['class'] == 'Iris-versicolor') &
              (iris_data['sepal_length_cm'] < 1.0)]

These near-zero `sepal_length_cm` entries seem to be off by two orders of magnitude, as if they had been recorded in meters instead of centimeters. After some brief correspondence with the field researchers, we find that one of them forgot to convert those measurements to centimeters. Let's do that for them.

In [ ]:
iris_data.loc[(iris_data['class'] == 'Iris-versicolor') &
              (iris_data['sepal_length_cm'] < 1.0),
              'sepal_length_cm'] *= 100.0

# Checking properties of the updated sepal_length_cm attribute for Iris versicolor
iris_data.loc[(iris_data['class'] == 'Iris-versicolor'),'sepal_length_cm'].describe()

#### 3. We have some rows with missing values.

Let's take a look at these rows:

In [ ]:
# Display all rows which contain missing values
iris_data.loc[(iris_data['sepal_length_cm'].isnull()) |
              (iris_data['sepal_width_cm'].isnull()) |
              (iris_data['petal_length_cm'].isnull()) |
              (iris_data['petal_width_cm'].isnull())]

It would not be ideal to exclude these rows considering they all belong to the `Iris-setosa` class. Since it seems like the missing data is systematic — all of the missing values are in the same column for the same *Iris* type — this error could potentially bias our analysis.

One way to deal with missing data is **mean imputation**: If we know that the values for a measurement fall in a certain range, we can fill in empty values with the average of that measurement. Besides the mean, using the median or zero can be reasonable choices. 

Let's use the mean here.

In [ ]:
# Display statistics of the petal_width_cm attribute for the Iris-setosa class
iris_data.loc[(iris_data['class'] == 'Iris-setosa'),'petal_width_cm'].describe()

Most of the petal widths for `Iris-setosa` fall within the 0.2-0.3 range. Let's fill in the missing entries with the average measured petal width.

In [ ]:
average_petal_width = iris_data.loc[iris_data['class'] == 'Iris-setosa', 'petal_width_cm'].mean()

# Fill missing Iris setosa petal width values with the mean value
iris_data.loc[(iris_data['class'] == 'Iris-setosa') &
              (iris_data['petal_width_cm'].isnull()),
              'petal_width_cm'] = average_petal_width

**Note:** Sometimes mean imputation might not be the best choice to deal with missing data, and we might want to remove incomplete rows. In such a case, you can drop all rows with missing data with the `dropna()` call:

    iris_data.dropna(inplace=True)

Now that we've cleaned the data, we don't want to repeat this process every time we work with the data set. Let's save the tidied data file *as a separate file* and work directly with that data file from now on.

In [ ]:
#Write cleaned data set to new csv (comma-separated values) file
iris_data.to_csv('iris-data-clean.csv', index=False)

# Load cleaned data into new DataFrame object
iris_data_clean = pd.read_csv('iris-data-clean.csv')

Before we go on, let's summarize some general takeaways related to tidying the data:

* Make sure the data is labeled properly
* Make sure the data falls within the expected range, and use domain knowledge whenever possible to define that expected range
* Deal with missing data: replace it if you can; otherwise drop it
* Never tidy the data manually because that is not easily reproducible; instead, use code as a record of how you modified the data

## Step 4: Exploratory analysis

[[ go back to the top ]](#Table-of-contents)

Exploratory analysis is the step where we start delving deeper into the data set beyond the outliers and errors. We'll be looking to answer questions such as:

* How is my data distributed?
* Are there any correlations in my data?
* Are there any confounding factors that explain these correlations?

This is the stage where we plot the data in as many ways as possible. Let's return to the scatterplot matrix that we used earlier and take a look at the cleaned data set:

In [ ]:
sb.pairplot(iris_data_clean)
;

Our data is normally distributed for the most part, which is great news if we plan on using any modeling methods that assume normally distributed data. But there's something strange going on with the petal measurements, which we suspect is related to the different `Iris` species. Let's color code the data by the class again:

In [ ]:
sb.pairplot(iris_data_clean, hue='class')
;

Indeed, the strange distribution of the petal measurements exists because of the different species. This is actually great news for our classification task since it means that the petal measurements will make it easy to distinguish between `Iris-setosa` and the other `Iris` types. Distinguishing `Iris-versicolor` and `Iris-virginica` will prove more difficult given the overlap in their measurements.

There are also correlations between petal length and petal width, as well as sepal length and sepal width. The field biologists assure us that this is to be expected: Longer flower petals also tend to be wider, and the same applies for sepals.

Now we have a better understanding of the data we are dealing with. Let's finally get to some modeling!

## Step 5: Classification

[[ go back to the top ]](#Table-of-contents)

You might be surprised that it took us so long to get to the actual modeling step. In general, data preparation tends to be by far the [most time-consuming](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/) step in data science, and it is a vital one: If we had jumped straight to the modeling, we would have created a faulty classification model. Remember to always check your data first: **Bad data leads to bad models.** 

Before we select and train a classification model, we need to **split the data into training and testing sets:**
- A **training set** is a random subset of the data that we use to train our models.
- A **testing set** is a random subset of the data (mutually exclusive from the training set) that we use to validate our models on data they have not seen before.

Especially in sparse data sets like ours, it's easy for models to **overfit** the data: The model will learn the training set so well that it won't be able to generalize and handle most of the cases it's never seen before. This is why it's important for us to build the model with the training set, but score it with the testing set.

Note that once we split the data into a training and testing set, we should treat the testing set like it no longer exists: We must use any information from the testing set to build our model or else we're cheating.

Let's set up our data first.

In [ ]:
# We're using all four measurements as inputs to the model
# Note that scikit-learn expects each entry to be a list of values, e.g.,
# [ [val1, val2, val3],
#   [val1, val2, val3],
#   ... ]
# such that our input data set is represented as a list of lists

# We can extract the data in this format from the DataFrame like this:
all_inputs = iris_data_clean[['sepal_length_cm', 'sepal_width_cm',
                             'petal_length_cm', 'petal_width_cm']].values
# Make sure to not mix up the order of the entries

# Similarly, we can extract the class labels
all_labels = iris_data_clean['class'].values

# Here's what a subset of our inputs looks like:
all_inputs[:5]

In [ ]:
# Checking that the representation is correct for a random input data point n
n = 42

print(all_inputs[n],all_labels[n])
iris_data_clean.loc[[n]]


Looks fine! Now our data is ready to be split. We split the data randomly using the method "train_test_split" from the **scikit-learn** library, the essential machine learning package in Python. We can specify what fraction of the data should be used for the test set using the parameter `test_size`:

In [ ]:
from sklearn.model_selection import train_test_split

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.25, random_state=1)

With our data split, we can start fitting models to our data. We will use a **decision tree classifier**: In their simplest form, decision tree classifiers ask a series of Yes/No questions about the data — each time getting closer to finding out the class of each entry — until they either classify the data set perfectly or simply can't differentiate a set of entries. 


Note: There are several [parameters](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) that we can tune for decision tree classifiers, but for now we use a basic decision tree classifier:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create the classifier
decision_tree_classifier = DecisionTreeClassifier()

# Train the classifier on the training set
decision_tree_classifier.fit(training_inputs, training_classes)

# Validate the classifier on the testing set using classification accuracy
decision_tree_classifier.score(testing_inputs, testing_classes)

That was easy: Our model achieves **97% classification accuracy** without much effort. However, there's a catch: The split of the data set was done randomly; depending on how our training and testing set is sampled, our model can achieve anywhere from 80% to 100% accuracy:

In [ ]:
model_accuracies = []

# Split data set randomly 1000 times, and train and test the model
for repetition in range(1000):
    (training_inputs,
     testing_inputs,
     training_classes,
     testing_classes) = train_test_split(all_inputs, all_labels, test_size=0.25)
    
    decision_tree_classifier = DecisionTreeClassifier()
    decision_tree_classifier.fit(training_inputs, training_classes)
    classifier_accuracy = decision_tree_classifier.score(testing_inputs, testing_classes)
    model_accuracies.append(classifier_accuracy)

# Display the accuracy distribution obtained    
plt.hist(model_accuracies)
plt.xlabel('model accuracy')
plt.ylabel('number of repetitions')
;

It's obviously a problem that our model performs quite differently depending on the subset of the data it's trained on. A better way to evaluate the model is to use **cross-validation**.

## Step 6: Evaluation with cross-validation

[[ go back to the top ]](#Table-of-contents)

A better approach to evaluate our model is to perform a ***k*-fold cross-validation**, which works as follows: Split the original data set into *k* subsets called *folds*; use one of the folds for evaluation, and train on the rest of the folds. Repeat *k* times such that each fold is used as the testing set exactly once. 

10-fold cross-validation is the most common choice. We can perform 10-fold cross-validation on our model with the code below. The result is an array containing the 10 evaluation scores:

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

decision_tree_classifier = DecisionTreeClassifier()
k = 10

# cross_val_score returns a list of the scores, which we can visualize
# to get a reasonable estimate of our classifier's performance
cv_scores = cross_val_score(decision_tree_classifier, all_inputs, all_labels, cv=k)

print(cv_scores)

Note that the `cross_val_score` method actually performs a **stratified *k*-fold cross-validation**. Stratified *k*-fold keeps the class proportions the same across all of the folds, which is vital for ensuring that the testing is performed on a representative subset of the data set.

Now we have a much more consistent rating of our classifier's general classification accuracy:

In [ ]:
plt.hist(cv_scores)
plt.title('Average score: {}'.format(np.mean(cv_scores)))
plt.xlabel('model accuracy')
plt.ylabel('number of folds')
;

Alright! We finally have our demo classifier in a complete and reproducible machine learning pipeline. We've met the success criteria that we set from the beginning (>90% accuracy), and our pipeline is flexible enough to handle new inputs or flowers when that data set is ready.

### <font color=blue>**It's your turn**</font> 

As a little exercise, let's investigate what result we would obtain for this classifier if we did not correct for the six outliers in the data. Using most of the code shown above, the steps you need to take are:

- Load the required libraries and 
- Import the data, adjust mislabeled classes and missing petal width values
- Save a new data set `iris-data-foo` without correcting the outliers 
- Extract the data into the format required by scikit-learn
- Train & evaluate your model using cross-validation 
- State the average accuracy obtained 

Before you start, take a moment to reflect: Would you expect the classification accuracy to deteriorate considerably? Why / why not?

In [ ]:
# Type your code here or create a new notebook



## Sources

[[ go back to the top ]](#Table-of-contents)

Based on a notebook by [Randal S. Olson](http://www.randalolson.com/). 

Sources for pictures:
* iris-types.png: https://gadictos.com/iris-data-classification-using-neural-net/
* iris_petal_sepal.png: https://holgerbrandl.github.io/kotlin4ds_kotlin_night_frankfurt/krangl_example_report.html

### Python packages used

This notebook uses several standard Python packages. These are:

* **pandas**: Provides the "DataFrame" structure to store data in memory and work with it easily and efficiently. DataFrame is a 2-dimensional labeled data structure with columns of potentially different types; you can think of it like a spreadsheet.
* **matplotlib**: Basic plotting library in Python; most other Python plotting libraries are built on top of it.
* **Seaborn**: Advanced statistical plotting library.
* **scikit-learn**: The essential Machine Learning package in Python.
* **NumPy**: Provides a fast numerical array structure and helper functions.